# Wrangling the data file

This notebook shows how the data was wrangled into a flat sheet for analysis.
Not all the columns are needed for analysis and they were not "varified" by the second reviewer as they were not used in analysis. Unreviewed columns include a \*:


## Data set 1: College EBP and Research Requirements

Column | What it refers to
:---|:------
ID | Unique entry ID
College_ID | Name of the college
Reserach_stage_ID | Research stage using Glaszious' Triangle
Competency_ID | Using Blooms Taxonomy
Compulsory_YES_ID|Whether the learning activity or assessment is repeated
Repeated_learning_YES_ID| Whether the learning activity or assessment is repeated
Activity | Text extracted from college documents
\* Source document | Where the text was extracted from
constructive_type | Whether the row is about a learning outcome, learning activity or assessment
scholarly_project | Whether the learning activity or assessment is associated with a scholarly project
activity_type|Type of learning activity or assessment activity (e.g. Scholarly Project)
LEARNINGA_research_training|Whether the learning activity is a form of formal research training
LEARNINGA_experienced_supervisor|Whether the scholarly project learning activity requires a research experienced supervisor: This was dispended, and category used instead.
LEARNINGA_experienced_supervisor_ID|Level of required research expertise by supervisor as described by the college
activity_type|Type of learning activity or assessment activity
\* RPL|Whether the learning activity or assessment has any recognition of prior learning
reviewed| Whether the entry has been reviewed by a second reviewer
discrepencies_resolved| Whether discrepencies have been resolved
exclude| Whether the row needs to be excluded from analysis
reason| descriptions of learning activity or assessment
project_category_ID| only option, points based or options based
HDR_option| Whether the scholarly project learning activity is a Higher Degree by Research(HDR) option or the assessment is associated with HDR: needs to be removed from analysis as we are interested in non-HDR project requirements
Assessment_format_ID | Assessment format category (e.g. publication, written exam, etc)
Criteria_YES_ID | Whether marking criteria was available for the assessment
Criteria_detail | Text extracted from college documents regarding marking critiera
HDR_option_avail|If the learning is a scholarly project, whether it can be completed as a HDR instead.


## Data set 2: College Duration

Column name| What it refers to
:---|:---
ID |Unique entry ID
Abbreviation | Abbreviated name of the college
Duration of training| Minimum duration of training
duration_max| Maximum duration of training
Comment| Any comments entered in by researcher



## Libraries

In [1]:
import pandas as pd

#import regex
import re

import numpy as np

import operator

----------------------

# Wrangling Dataset 2: college_learning_activity

In [2]:
#Read the workbook

df = pd.read_excel('Data/college_name_learning_activity.xlsx')

#df.head()

In [3]:
print("Data frame shape is: ", df.shape)
print(df.info())


#df.head()

('Data frame shape is: ', (978, 27))
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 978 entries, 0 to 977
Data columns (total 27 columns):
ID1                                    978 non-null int64
ID                                     977 non-null float64
College_ID                             977 non-null object
Reserach_stage_ID                      977 non-null object
Competency_ID                          977 non-null object
Compulsory_YES_ID                      475 non-null object
Repeated_learning_YES_ID               475 non-null object
Activity                               977 non-null object
Source document                        977 non-null object
constructive_type                      977 non-null object
scholarly_project                      978 non-null bool
activity_type                          439 non-null object
LEARNINGA_research_training            978 non-null bool
LEARNINGA_experienced_supervisor       978 non-null bool
LEARNINGA_experienced_supervisor_ID   

In [4]:
#required columns only
df1 =  df[['College_ID',
           "Reserach_stage_ID",
           "Competency_ID", 
           "constructive_type",
           "scholarly_project",
           "HDR_option",
           "HDR_option_avail",
           "Repeated_learning_YES_ID",
           "Compulsory_YES_ID",
           "project_category_ID",
           "RPL",
           "LEARNINGA_research_training",
           "LEARNINGA_experienced_supervisor",
           "LEARNINGA_experienced_supervisor_ID",
           "activity_type",
           "reviewed",
           "discrepencies_resolved",
           'exclude',
          ]]

print("Data frame shape is:", df1.shape)

#df1.head()

('Data frame shape is:', (978, 18))


In [5]:
#only include those that have been reviewed and discrepencies resolved
#exclude rows that have been "excluded"
#also want to exclude learning and assessment activities associated with HDR

df_included = df1[(df1['exclude'] == False) & 
                  (df1['reviewed'] == True) & 
                  (df1["discrepencies_resolved"] == True) &
                 (df1["HDR_option"] == False)]

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (797, 18))


In [6]:
#Remove all joint colleges

df_included = df_included[~df_included .College_ID.str.match("RACP/")]


print("Data frame shape is: ", df_included.shape)
#df_included.head()


('Data frame shape is: ', (777, 18))


In [7]:
#Change name of CICM (paediatric)and CICM(adult) to CICM 
#CICM(adult) and CICM(paediatric) had the same handbook but different LO for both (realised after intital entry)
#need to merge

#First check that there are no learning or assessment activities in CICM(paeds)

df_included[(df_included['College_ID'] == "CICM (paediatric)") & 
                  (df_included["constructive_type"] != "learning_outcome")]


,College_ID,Reserach_stage_ID,Competency_ID,constructive_type,scholarly_project,HDR_option,HDR_option_avail,Repeated_learning_YES_ID,Compulsory_YES_ID,project_category_ID,RPL,LEARNINGA_research_training,LEARNINGA_experienced_supervisor,LEARNINGA_experienced_supervisor_ID,activity_type,reviewed,discrepencies_resolved,exclude


In [8]:
#also check that the learning ojectives for CICM (paediatric) are there

#df_included[(df_included['College_ID'] == "CICM (paediatric)") & 
                  #(df_included["constructive_type"] == "learning_outcome")]

In [9]:
#Change name of CICM (paediatric) and CICM(adult) to CICM 

df_included.loc[(df_included.College_ID.str.match("CICM")), "College_ID"] = 'CICM'
#df_included.loc[df_included.College_ID =='CICM (paediatric)'] = 'CICM'


print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (777, 18))


## Consistent naming
Some of the abbreviated college names were entered inconsistently; need to change for consistancy across dataset

In [10]:
#Change name of colleges for consistency

df_included.loc[df_included.College_ID =='ACD (Dermatology)',"College_ID"] = 'ACD'
df_included.loc[df_included.College_ID =='RANZCP (Psychiatry)', "College_ID"] = 'RANZCP'
df_included.loc[df_included.College_ID =='RANZCR (rad onc)', "College_ID"] = 'RANZCR: Radiation Oncology'

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (777, 18))


In [11]:
len(set(df_included.College_ID))

61

## Add "all" data to respective colleges

Some colleges had over arching curriculula:
- RACP: ALL applies to all RACP colleges
- RCPA: basic applies to all RCPA colleges
- RACS: ALL applies to all RACS colleges

We need to make a copy of these over arching data points and add to the individual colleges and then remove RACP: All, RCPA: basic and RACS: ALL.


### RACP

In [12]:
#get a list of all the RACP colleges

#get those that have RACP: at the beginning
#exclude those that are part of RACP: All
#get the college ID (name) column
#create a unique list of college names
racp_colleges = df_included[(df_included.College_ID.str.match("RACP:") )& (df_included.College_ID != "RACP: All")].College_ID.unique()

#racp_colleges

In [13]:
#Create a data frame of  RACP:ALL
# this will need to be copied over to each of the RACP colleges

racp_all = df_included[df_included.College_ID.str.contains("RACP: ALL")]

#racp_all.head()

In [14]:
#append copied data
for i in racp_colleges:
    
    #change the name to the correct college
    racp_all.College_ID = i
    
    #append the data to our included dataframe
    df_included = df_included.append(racp_all)

#remove the college RACP:ALL
df_included = df_included[df_included.College_ID != "RACP: ALL"]

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1096, 18))


C:\Users\pstehlik\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


### RCPA

In [15]:
#get a list of all the RCPA colleges

#get those that have RCPA: at the beginning
#exclude those that are part of RCPA: basic
#get the college ID (name) column
#create a unique list of college names
rcpa_colleges = df_included[(df_included.College_ID.str.match("RCPA") )& (df_included.College_ID != "RCPA: basic")].College_ID.unique()

#rcpa_colleges 

In [16]:
#Create a data frame of  RCPA:basic
# this will need to be copied over to each of the RCPA colleges

rcpa_all = df_included[df_included.College_ID.str.contains("RCPA: basic")]

#rcpa_all.head()

In [17]:
#append copied data
for i in rcpa_colleges:
    
    #change the name to the correct college
    rcpa_all.College_ID = i
    
    #append the data to our included dataframe
    df_included = df_included.append(rcpa_all)

#remove the college RACP:ALL
df_included = df_included[df_included.College_ID != "RCPA: basic"]

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1105, 18))


### RACS

In [18]:
#get a list of all the RACS colleges

#get those that have RACS: at the beginning
#exclude those that are part of RACS: All
#get the college ID (name) column
#create a unique list of college names
racs_colleges = df_included[(df_included.College_ID.str.match("RACS") )& (df_included.College_ID != "RACS: ALL")].College_ID.unique()

#racs_colleges    

In [19]:
#Create a data frame of  RACS: ALL
# this will need to be copied over to each of the RACS colleges

racs_all = df_included[df_included.College_ID.str.contains("RACS: ALL")]

#racs_all.head()

In [20]:
#append copied data
for i in racs_colleges:
    
    #change the name to the correct college
    racp_all.College_ID = i
    
    #append the data to our included dataframe
    df_included = df_included.append(racs_all)

#remove the college RACP:ALL
df_included = df_included[df_included.College_ID != "RACS: ALL"]

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1098, 18))


In [21]:
len(set(df_included.College_ID))

58

## Add college number 

Add college number column that matches college with number 

In [22]:
#create df of college names and numbers

df_numbers = df_included[["College_ID"]]

#drop duplicate college names
df_numbers.drop_duplicates(subset ="College_ID", 
                     keep = "first", inplace = True) 

#order alphabetically
df_numbers = df_numbers.sort_values('College_ID')

print("Data frame shape is: ", df_numbers.shape)
#df_numbers.head()


('Data frame shape is: ', (58, 1))


C:\Users\pstehlik\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
#create a column to give a number to represent each college

df_numbers["college_number"] = np.arange(len(df_numbers)) + 1


#df_numbers.head()

In [24]:
#df_numbers

In [25]:
#create a new column

df_included["college_number"] = df_included["College_ID"]

#df_included.head()

In [26]:
#https://stackoverflow.com/questions/20250771/remap-values-in-pandas-column-with-a-dict

#convert df into dictionary
numbers = dict(zip(df_numbers.College_ID, df_numbers.college_number))

#replace names with numbers
df_included["college_number"].replace(numbers, inplace = True)


print("Data frame shape is: ", df_included.shape)
#df_included.head()



('Data frame shape is: ', (1098, 19))


# Add "clean" category columns

Add constructive type abbreviation column just to make life easier later

In [27]:
#create a new column

df_included ["constructive_type_abv"] = np.nan

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1098, 20))


In [28]:
#add abbreviation group names based on constructive type column

df_included ['constructive_type_abv'] = pd.np.where(df_included.constructive_type.str.match("learning_outcome"), "Objective",
                                                    pd.np.where(df_included .constructive_type.str.match("learning_activity"), "Activity",
                                                                pd.np.where(df_included.constructive_type.str.match("assessment_activity"), "Assessment", np.nan)))

print("Data frame shape is: ", df_included.shape)
#df_included .head()

('Data frame shape is: ', (1098, 20))


In [29]:
#check that it worked


print("Original: ", df_included.constructive_type.value_counts())


print("Clean: ",df_included.constructive_type_abv.value_counts())

('Original: ', learning_outcome       684
assessment_activity    256
learning_activity      158
Name: constructive_type, dtype: int64)
('Clean: ', Objective     684
Assessment    256
Activity      158
Name: constructive_type_abv, dtype: int64)


Same goes for research level column

In [30]:
#create a new column

df_included ["Research Level"] = np.nan

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1098, 21))


In [31]:
df_included ["Research Level"] = pd.np.where(df_included.Reserach_stage_ID .str.contains("EBP"), "Using",
                                                    pd.np.where(df_included .Reserach_stage_ID .str.contains("Participating"), "Participating",
                                                                pd.np.where(df_included.Reserach_stage_ID .str.contains("Leading"), "Leading", "Undef")))

print("Data frame shape is: ", df_included.shape)
#df_included 

('Data frame shape is: ', (1098, 21))


In [32]:
#check that it worked


print("Original: ", df_included.Reserach_stage_ID.value_counts())


print("Clean: ",df_included["Research Level"].value_counts())


('Original: ', 3 - \tLeading research              548
1 - \tUsing research (EBP)          496
2 - \tParticipating in research      53
2 - \tParticipating in research)      1
Name: Reserach_stage_ID, dtype: int64)
('Clean: ', Leading          548
Using            496
Participating     54
Name: Research Level, dtype: int64)


And competency level column

In [33]:
#create a new column

df_included ["Competency"] = np.nan


#df_included.head()

In [34]:
df_included ["Competency"] = pd.np.where(df_included.Competency_ID .str.contains("1 - Remembering"), "Remembering",
                                                    pd.np.where(df_included .Competency_ID .str.contains("2 - Understanding"), "Understanding",
                                                                pd.np.where(df_included.Competency_ID .str.contains("3 - Applying"), "Applying",
                                                                            pd.np.where(df_included.Competency_ID .str.contains("4 - Analysing"), "Analysing",
                                                                                        pd.np.where(df_included.Competency_ID .str.contains("5 - Evaluating"), "Evaluating",
                                                                                                    pd.np.where(df_included.Competency_ID .str.contains("6 - Creating"), "Creating","Undef"))))))

print("Data frame shape is: ", df_included.shape)
#df_included.head()

('Data frame shape is: ', (1098, 22))


In [35]:
#check that it worked


print("Original: ", df_included.Competency_ID.value_counts())


print("Clean: ",df_included.Competency.value_counts())


('Original: ', 6 - Creating         494
5 - Evaluating       257
2 - Understanding    161
3 - Applying         110
1 - Remembering       53
4 - Analysing         23
Name: Competency_ID, dtype: int64)
('Clean: ', Creating         494
Evaluating       257
Understanding    161
Applying         110
Remembering       53
Analysing         23
Name: Competency, dtype: int64)


## Create de-identified dataframe

In [36]:
df_included.reviewed.unique()

array([ True])

In [37]:
df_included.discrepencies_resolved.unique()

array([ True])

In [38]:
df_included.exclude.unique()

array([False])

In [39]:
df_deidenfified = df_included[[
        "college_number",
        #'College_ID',
         #  "Reserach_stage_ID",
    "Research Level",
          # "Competency_ID", 
    "Competency",
         #  "constructive_type",
    "constructive_type_abv",
           "scholarly_project",
           "HDR_option",
           "HDR_option_avail",
           "Repeated_learning_YES_ID",
           "Compulsory_YES_ID",
           "project_category_ID",
           "RPL",
           "LEARNINGA_research_training",
           "LEARNINGA_experienced_supervisor",
           "LEARNINGA_experienced_supervisor_ID",
           "activity_type",
          # "reviewed",
          # "discrepencies_resolved",
          # 'exclude'
          #"Year_achieved_ID"
          ]]

#rename college_numbers to College_ID
df_deidenfified = df_deidenfified.rename(columns={'college_number': 'College_ID'})


df_deidenfified.head()

,College_ID,Research Level,Competency,constructive_type_abv,scholarly_project,HDR_option,HDR_option_avail,Repeated_learning_YES_ID,Compulsory_YES_ID,project_category_ID,RPL,LEARNINGA_research_training,LEARNINGA_experienced_supervisor,LEARNINGA_experienced_supervisor_ID,activity_type
0,38,Leading,Creating,Activity,True,False,False,2 - No,1 - Yes,1 - Only option,True,False,False,4 - Experienced research supervisor ENCOURAGED,Scholalry Project
1,37,Leading,Creating,Activity,True,False,True,2 - No,1 - Yes,1 - Only option,False,False,False,2 - Supervisor required but no details given,Scholalry Project
2,38,Using,Evaluating,Activity,False,False,False,2 - No,1 - Yes,1 - Only option,False,True,False,NaN,CLEAR course
3,37,Using,Evaluating,Activity,False,False,False,2 - No,1 - Yes,1 - Only option,False,True,False,NaN,CLEAR course
4,36,Leading,Creating,Activity,True,False,True,2 - No,1 - Yes,1 - Only option,True,False,False,2 - Supervisor required but no details given,Scholalry Project


df_included.info()

## Save csv file


In [40]:
#save as csv
df_deidenfified.to_csv('Outputs/included_data.csv', sep=',', encoding='utf-8')

-------

# Wrangling Dataset 1: college_duration

## Changing duration of training for RACP Colleges

- All RACP colleges have a 3 year basic component of their training, followed by the duration of the adult training.
- .: we need to add the duration of basic training to the advance training as this was not done previously.

In [41]:
#Read the workbook
duration = pd.read_excel('Data/college_duration.xlsx')
#duration.head()

In [42]:
print("Data frame shape is: ", duration.shape)
print(duration.info())

('Data frame shape is: ', (69, 5))
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 5 columns):
ID                      69 non-null int64
Abbreviation            69 non-null object
Duration of training    68 non-null float64
duration_max            69 non-null object
Comments                56 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 2.8+ KB
None


In [43]:
#Remove all joint colleges

duration = duration[~duration.Abbreviation.str.match("RACP/")]


print("Data frame shape is: ", duration.shape)
#duration.head()


('Data frame shape is: ', (65, 5))


In [44]:
#Change name of CICM (paediatric) and CICM(adult) to CICM 

duration.loc[(duration.Abbreviation.str.match("CICM")), "Abbreviation"] = 'CICM'
#df_included.loc[df_included.College_ID =='CICM (paediatric)'] = 'CICM'


print("Data frame shape is: ", duration.shape)
#duration.head()


('Data frame shape is: ', (65, 5))


In [45]:
#drop duplicates

# dropping ALL duplicte values 
duration.drop_duplicates(subset ="Abbreviation", 
                     keep = "first", inplace = True) 

print("Data frame shape is: ", duration.shape)
#duration.head()


('Data frame shape is: ', (64, 5))


In [46]:
#Change name of colleges for consistency

duration.loc[duration.Abbreviation =='ACD (Dermatology)',"Abbreviation"] = 'ACD'
duration.loc[duration.Abbreviation =='RANZCP (Psychiatry)', "Abbreviation"] = 'RANZCP'
duration.loc[duration.Abbreviation =='RANZCR (rad onc)', "Abbreviation"] = 'RANZCR: Radiation Oncology'

#duration.head()

In [47]:
#remove the "ALL" colleges

#remove the college RACP:ALL
duration = duration[duration.Abbreviation != "RACP: ALL"]

#remove the college RCPA: basic
duration = duration[duration.Abbreviation != "RCPA: basic"]

#remove the college RACS: ALL
duration = duration[duration.Abbreviation != "RACS: ALL"]

print("Data frame shape is: ", duration.shape)
#duration.head()

('Data frame shape is: ', (61, 5))


In [48]:
#remove RACP basic trainings
duration = duration[~duration.Abbreviation.str.match("RACP: Basic")]

In [49]:
#add 3 years to all RACP colleges

duration.loc[duration.Abbreviation.str.match("RACP"), "Duration of training"] = duration["Duration of training"] + 3


#duration.head()


In [50]:
#create a column to give a number to represent each college

duration["college_number"] = duration["Abbreviation"]

#replace names with numbers
duration["college_number"].replace(numbers, inplace = True)


print("Data frame shape is: ", duration.shape)
#duration.head()

('Data frame shape is: ', (59, 6))


In [51]:
#what is the largest college number
i = max(numbers.items(), key=operator.itemgetter(1))[1] +1

array = list(range(0, i))


#get dataframe of colleges without a numbers
un_numbered = duration.loc[~duration["college_number"].isin(array)]

#replace college_number college name in with number 

for college in un_numbered["Abbreviation"]:
    duration.loc[duration['Abbreviation'] == college, 'college_number'] = i
    i = 1+i

In [52]:
#create new dataframe with required columns

min_duration = duration[[
    "college_number",
    "Duration of training"]]



#rename abbreviation to College
min_duration = min_duration.rename(index=str, columns={"college_number": "College"})

print(min_duration.shape)
min_duration.head()


(59, 2)


,College,Duration of training
0,1,4.0
1,2,3.5
2,3,5.0
3,4,6.0
5,5,3.0


In [53]:
#save as csv
min_duration.to_csv('Outputs/min_duration.csv', sep=',', encoding='utf-8')